# Corporate FinTech - Assignment 1

In [39]:
#%% Imports
#import math
import numpy as np
#import statistics
import pandas as pd
#import scipy.stats as scs
# from pylab import mpl, plt
import random
from matplotlib.pylab import plt#, mpl
from datetime import datetime
from dateutil.relativedelta import relativedelta
import scipy.optimize as sco
import warnings
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import webbrowser

ModuleNotFoundError: No module named 'pypfopt'

In [2]:
#%% Name and e-mail
print('Mads Duelund Dorka, mador17@student.sdu.dk')
print('Max Festersen Hansen, maxfh20@student.sdu.dk')
print('Mathias Eriksen, merik17@student.sdu.dk')
print('Daniel Lindberg, dlind16@student.sdu.dk')
print('Emilie Bruun Therp, emthe15@student.sdu.dk')

Mads Duelund Dorka, mador17@student.sdu.dk
Max Festersen Hansen, maxfh20@student.sdu.dk
Mathias Eriksen, merik17@student.sdu.dk
Daniel Lindberg, dlind16@student.sdu.dk
Emilie Bruun Therp, emthe15@student.sdu.dk


In [5]:
warnings.filterwarnings('ignore')

NameError: name 'warnings' is not defined

In [6]:
#%% Design
plt.style.use('fivethirtyeight')

In [33]:
#%% Read data into code
data = pd.read_csv('https://www.maxfest.dk/sdu/crypto-markets.csv')
#data.describe()

new_data = data.loc[data['date'] > '2017 - 01 - 01'] # Sort date
#new_data.info()

# Exercise 1

In [34]:
webbrowser.open('https://www.maxfest.dk/sdu/ex1.pdf')  # Go to example.com

NameError: name 'webbrowser' is not defined

# Exercise 2

In [ ]:
#%% Exercise 2 preperation
random.seed(42069)
results = { # generate dictionary to carry information for each iteration
    "i" : [], # Iteration
    "N": [], # Number of cryptos
    "portfolio_return" : [],
    "portfolio_volatility" : [],
    "sharpe_ratio" : []
}
results["i"].append(0) # First result is created outside of loop

# Set inital data
pd_data = new_data[['date', 'name', 'close']] # Filter to date, name and close

# Preperation to choose random date
unique_days = pd_data["date"].unique() # filter unique days in data
last_day = datetime.strptime(unique_days[-1], "%Y-%m-%d") # Get last date entry in date format (to make date calculations)
last_available_date = last_day - relativedelta(months=+6) # Get last date that is at least 6 months from end date
last_available_date = last_available_date.strftime("%Y-%m-%d") # Format as string
unique_days_filter = unique_days <= last_available_date # create filter list
unique_days = unique_days[unique_days_filter] # filter unavailable dates (not with 6 months after)
n_unique_days = len(unique_days) # count unique days

## 1.) 
Rank cryptos according to their median overall trading volume.

In [ ]:
new_data_1 = new_data[['date', 'name', 'volume']] # Get data needed

In [ ]:
# Use to print out nicely :D
new_data_median_sorted = new_data_1.groupby("name").agg({"volume": ["median"]}).sort_values(by=("volume", "median"), ascending=False) # Group by name, aggregate volume by median and sort by volume as primary and then median as secondary
# print(new_data_median_sorted)
crypto_curr = new_data_median_sorted.head(50) # Get 50 first results - the top 50 because of sorting
print(crypto_curr) # Print results
# 2017-06-01 - 2018-01-01 (Close values)
print("PlaceholderText")

## 2.) 
Use the 50 most traded cryptos according to your ranking in 1.) 

### 2.2.a)
Randomly choose a date and randomly choose N cryptos on this date.

In [11]:
# Choosing a random date
n_randdom_day = random.randint(0,n_unique_days-1) # Select random entry number
unique_day = unique_days[n_randdom_day] # get entry
unique_day_until = datetime.strptime(unique_day, "%Y-%m-%d") + relativedelta(months=+6) # Find 6 months after random date
unique_day_until = unique_day_until.strftime("%Y-%m-%d") # Format as string

In [12]:
# Print random date information:
print("The random chosen date is: " + unique_day)

The random chosen date is: 2017-04-03


In [13]:
# Choose a random N of cryptos
random_top_curr = crypto_curr.index.values # top 50 curencies - from 2.1
for n in range(random.randint(0,49)): # Randomly run from 0 to 49 times (so at least 1 value is left)
    random_top_curr = np.delete(random_top_curr, random.randint(0,len(random_top_curr)-1), 0) # remove a random value

In [14]:
weight_random_top_curr = len(random_top_curr) # Amount of currencsies

In [15]:
print("There are ", weight_random_top_curr, " currencies.\nThe random top currencies are:")
print(random_top_curr)

There are  9  currencies.
The random top currencies are:
['Bitcoin' 'Tether' 'Ethereum' 'EOS' 'OmiseGO' 'Monero' 'Arcblock' '0x'
 'Status']


In [16]:
# Filter the days
is_dates = pd_data['date'] >= unique_day # create filtering list from random date
pd_data = pd_data[is_dates] # Filter by the random date and after
is_dates = pd_data['date'] <= unique_day_until # create filtering list until 6 months after random date
pd_data = pd_data[is_dates] # Filter by the random date and after

In [38]:
# Filter by top currensies
curr_in = pd_data['name'].isin(random_top_curr) # Create filter list by random top currencies
pd_data = pd_data[curr_in] # Apply filter list

### 2.2.b)
Use 1/N weights and calculate the corresponding portfolio return, portfolio volatility, and Sharpe ratio for a holding period of up to six calendar months.

In [18]:
pd_data_1 = pd_data.pivot_table(index = 'date', columns = 'name', values = 'close') # Set names as columns, close as values with date as index
ret = np.log(pd_data_1/pd_data_1.shift(1)) # Calculate log-return of different stocks

In [19]:
ret = ret.dropna(axis=1, how='all').dropna(axis=0, how='any')  # Handling missing values, drop any column with a nan
for item in random_top_curr:
    if(item not in ret.columns):
        print(item + " removed because it was empty for holding period.")

Arcblock removed because it was empty for holding period.


In [20]:
if (len(ret.columns) < weight_random_top_curr):
    print("N is now: " + str(len(ret.columns)))

N is now: 8


In [21]:
weight = np.array(1/len(ret.columns)) # calculate 1/N weights

In [22]:
results["N"].append(len(ret.columns)) # Append Number of columns to results dictionary

In [23]:
weight_array = np.full((len(ret.columns), 1), weight) # Create array with weights

In [24]:
# Portfolio return, Portfolio volatility, and Sharpe ratio
# Portfolio volatility (for holding period)
#ret.mean() * 126 # Calculate mean of return (from half of an year)
#ret.cov() * 126 # Calculate covariance matrix for stocks

#np.sum(ret.mean().values * weight_array) * 126
#np.dot(weight_array.T, np.dot(ret.cov() * 126, weight_array))
#math.sqrt(np.dot(weight_array.T, np.dot(ret.cov() * 126, weight_array )))

In [25]:
def port_ret(weight_array):
    return np.sum(np.array(ret.mean().values) * weight_array) * 126

In [26]:
def port_vol(weight):
    p_var = np.dot(
               weight.T,
               ret.cov() * 126
           )
    p_var = np.sqrt(p_var)
    p_var[np.isnan(p_var)] = 0
    return(p_var)

In [27]:
# Add to array (for ex2 c and ex2 d)
results["portfolio_return"].append(port_ret(weight_array))
results["portfolio_volatility"].append(np.sum(port_vol(weight_array)))

<ipython-input-26-80b850912364>:6: RuntimeWarning: invalid value encountered in sqrt
  p_var = np.sqrt(p_var)


In [28]:
# Sharpe ratio (for holding period)
Sharpe_Ratio = ret.mean()/ret.std()
Holding_SR = (126**0.5) * sum(Sharpe_Ratio)
results["sharpe_ratio"].append(Holding_SR)

### 2.2.c)
repeat a.) and b.) 10,000 times and provide descriptive statistics for the resulting portfolio return, portfolio volatility, and Sharpe ratio for the number of cryptos N = 1, 2, …, 50.

In [36]:
for i in range(1,10000):
    results["i"].append(i)
    # Set inital data
    pd_data = new_data[['date', 'name', 'close']]  # Filter to date, name and close

In [37]:
# Filtering
# Choosing a random date
    n_randdom_day = random.randint(0, n_unique_days - 1)  # Select random entry number
    unique_day = unique_days[n_randdom_day]  # get entry
    unique_day_until = datetime.strptime(unique_day, "%Y-%m-%d") + relativedelta(months=+6)  # Find 6 months after random date
    unique_day_until = unique_day_until.strftime("%Y-%m-%d")  # Format as string

IndentationError: unexpected indent (<ipython-input-37-4682ecd36e52>, line 3)

In [ ]:
# Choose a random N of cryptos
    random_top_curr = crypto_curr.index.values  # top 50 curencies - from 2.1
    for n in range(random.randint(0, 49)):  # Randomly run from 0 to 49 times (so at least 1 value is left)
        random_top_curr = np.delete(random_top_curr, random.randint(0, len(random_top_curr) - 1), 0)  # remove a random value

In [ ]:
    weight_random_top_curr = len(random_top_curr)  # Amount of currencsies

In [ ]:
# Filter the days
    is_dates = pd_data['date'] >= unique_day  # create filtering list from random date
    pd_data = pd_data[is_dates]  # Filter by the random date and after
    is_dates = pd_data['date'] <= unique_day_until  # create filtering list until 6 months after random date
    pd_data = pd_data[is_dates]  # Filter by the random date and after

In [ ]:
 # Filter by top currensies
    curr_in = pd_data['name'].isin(random_top_curr)  # Create filter list by random top currencies
    pd_data = pd_data[curr_in]  # Apply filter list

In [ ]:
# Formating and calculations
    pd_data_1 = pd_data.pivot_table(index='date', columns='name', values='close')  # Set names as columns, close as values with date as index
    ret = np.log(pd_data_1 / pd_data_1.shift(1))  # Calculate log-return of different stocks

In [ ]:
 ret = ret.dropna(axis=1, how='all').dropna(axis=0, how='all')  # Handling missing values, drop any column with a nan
    for name, values in ret.cov().iteritems():
        curr = values.values
        curr[np.isnan(values)] = 0
        if sum(curr) == 0:
            ret = ret.drop(columns=[name])
    if len(ret.columns) == 0: # if there are no columns set vals as NaN
        results["N"].append(0)
        results["portfolio_return"].append(0)
        results["portfolio_volatility"].append(0)
        results["sharpe_ratio"].append(0)
    else:
        weight = np.array(1 / len(ret.columns))  # calculate 1/N weights
        results["N"].append(len(ret.columns))

In [ ]:
    weight_array = np.full((len(ret.columns), 1), weight)

In [ ]:
# Portfolio return, Portfolio volatility, and Sharpe ratio
        # Portfolio return (for holding period)
        results["portfolio_return"].append(port_ret(weight_array))

In [ ]:
 # Portfolio volatility (for holding period)
        results["portfolio_volatility"].append(np.sum(port_vol(weight_array)))

In [ ]:
# Sharpe ratio (for holding period)
        Sharpe_Ratio = ret.mean() / ret.std()
        Holding_SR = (126 ** 0.5) * sum(Sharpe_Ratio)
        results["sharpe_ratio"].append(Holding_SR)

In [ ]:
print("The code ran 10000 times.")

### 2.2.d)
Plot the average portfolio volatility against the number of cryptos N in the portfolio.

In [ ]:
ex2 = plt.scatter(results["N"], results["portfolio_volatility"])
#ex2_return = plt.scatter(results["N"], results["portfolio_return"])
plt.show()
print("PlaceholderText")

## 3.)
Repeat the procedure in 2. but instead of using 1/N weights use Sharpe ratio maximizing weights from classical portfolio optimization (conditional on N ≥ 4).

### 3.3.a)

In [ ]:
results_2_3 = { # generate dictionary to carry information for each iteration
    "i" : [], # Iteration
    "N": [], # Number of cryptos
    "portfolio_return" : [],
    "portfolio_volatility" : [],
    "sharpe_ratio" : []
}
results_2_3["i"].append(0) # First result is created outside of loop

In [ ]:
# Set inital data
pd_data = new_data[['date', 'name', 'close']] # Filter to date, name and close

In [ ]:
new_data_1 = new_data[['date', 'name', 'volume']] # Get data needed

In [ ]:
# Choosing a random date
n_randdom_day = random.randint(0,n_unique_days-1) # Select random entry number
unique_day = unique_days[n_randdom_day] # get entry
unique_day_until = datetime.strptime(unique_day, "%Y-%m-%d") + relativedelta(months=+6) # Find 6 months after random date
unique_day_until = unique_day_until.strftime("%Y-%m-%d") # Format as string

In [ ]:
# Print random date information:
print("The random chosen date is: " + unique_day)

In [ ]:
# Choose a random N of cryptos
random_top_curr = crypto_curr.index.values # top 50 curencies - from 2.1
for n in range(random.randint(0,49)): # Randomly run from 0 to 49 times (so at least 1 value is left)
    random_top_curr = np.delete(random_top_curr, random.randint(0,len(random_top_curr)-1), 0) # remove a random value

In [ ]:
weight_random_top_curr = len(random_top_curr) # Amount of currencsies

In [ ]:
print("There are ", weight_random_top_curr, " currencies.\nThe random top currencies are:")
print(random_top_curr)

In [ ]:
# Filter the days
is_dates = pd_data['date'] >= unique_day # create filtering list from random date
pd_data = pd_data[is_dates] # Filter by the random date and after
is_dates = pd_data['date'] <= unique_day_until # create filtering list until 6 months after random date
pd_data = pd_data[is_dates] # Filter by the random date and after

In [ ]:
# Filter by top currencies
curr_in = pd_data['name'].isin(random_top_curr) # Create filter list by random top currencies
pd_data = pd_data[curr_in] # Apply filter list

In [ ]:
pd_data_1 = pd_data.pivot_table(index = 'date', columns = 'name', values = 'close') # Set names as columns, close as values with date as index
ret = pd_data_1/pd_data_1.shift(1) # Calculate log-return of different stocks

In [ ]:
ret = ret.dropna(axis=1, how='all').dropna(axis=0, how='all')  # Handling missing values, drop any column with a nan
for item in random_top_curr:
    if(item not in ret.columns):
        print(item + " removed because it was empty for holding period.")

In [ ]:
if (len(ret.columns) < weight_random_top_curr):
    print("N is now: " + str(len(ret.columns)))

### 3.3.b)

In [ ]:
results_2_3["N"].append(len(ret.columns)) # Append Number of columns to results dictionary

In [ ]:
# Sharpe ratio (for holding period)
Sharpe_Ratio = ret.mean()/ret.std() # Calculate sharpe ratio for each currency
Holding_SR = (126**0.5) * Sharpe_Ratio # Account for holding period

In [ ]:
weight = np.array(1/len(ret.columns)) # Calculate 1/N weights
weight_array = np.full((len(ret.columns), 1), weight)  # Create array with weight N times

In [ ]:
def min_func_sharpe(weight_array):
    print("her")
    print(weight_array)
    min_sharpe = -port_ret(weight_array) / np.sum(port_vol(weight_array))
    return(min_sharpe)

In [ ]:
cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})
bnds = tuple((0, 1) for x in range(len(ret.columns)))

In [ ]:
opts = sco.minimize(min_func_sharpe, weight, method='SLSQP', bounds=bnds, constraints=cons)
weight_array = opts["x"].T

In [ ]:
# Portfolio return, Portfolio volatility, and Sharpe ratio
# Add to array (for ex2.3 c and ex2.3 d)
results_2_3["portfolio_return"].append(port_ret(weight_array))
results_2_3["portfolio_volatility"].append(np.sum(port_vol(weight_array)))
results_2_3["sharpe_ratio"].append(port_ret(weight_array)/np.sum(port_vol(weight_array)))

In [ ]:
print("The loop for Excersice 2.3.c did not work, thus 2.3.c-d and 2.4 could not be done. We used the example from the lectures as a base. During our handling of invalid data, and reformating to run the code, somehow it lost the ability to run.")

### 3.3.c)

In [ ]:
#for i in range(1,10000):
for i in range(1,5):
    # Set inital data
    pd_data = new_data[['date', 'name', 'close']]  # Filter to date, name and close

In [ ]:
# Choosing a random date
    n_randdom_day = random.randint(0, n_unique_days - 1)  # Select random entry number
    unique_day = unique_days[n_randdom_day]  # get entry
    unique_day_until = datetime.strptime(unique_day, "%Y-%m-%d") + relativedelta(months=+6)  # Find 6 months after random date
    unique_day_until = unique_day_until.strftime("%Y-%m-%d")  # Format as string

In [ ]:
# Choose a random N of cryptos
    random_top_curr = crypto_curr.index.values  # top 50 curencies - from 2.1
    for n in range(random.randint(0, 49)):  # Randomly run from 0 to 49 times (so at least 1 value is left)
        random_top_curr = np.delete(random_top_curr, random.randint(0, len(random_top_curr) - 1), 0)  # remove a random value

In [ ]:
weight_random_top_curr = len(random_top_curr)  # Amount of currencsies

In [ ]:
# Filter the days
    is_dates = pd_data['date'] >= unique_day  # create filtering list from random date
    pd_data = pd_data[is_dates]  # Filter by the random date and after
    is_dates = pd_data['date'] <= unique_day_until  # create filtering list until 6 months after random date
    pd_data = pd_data[is_dates]  # Filter by the random date and after

In [ ]:
# Filter by top currencies
    curr_in = pd_data['name'].isin(random_top_curr)  # Create filter list by random top currencies
    pd_data = pd_data[curr_in]  # Apply filter list
    pd_data_1 = pd_data.pivot_table(index='date', columns='name', values='close')  # Set names as columns, close as values with date as index
    ret = pd_data_1 / pd_data_1.shift(1)  # Calculate log-return of different stocks
    ret = ret.dropna(axis=1, how='all').dropna(axis=0, how='all')  # Handling missing values, drop any column with a nan

In [ ]:
    for name, values in ret.cov().iteritems(): # Drop values that have no cov
        curr = values.values # Get values
        curr[np.isnan(values)] = 0 # Set to 0
        if sum(curr) == 0:
            ret = ret.drop(columns=[name])

In [ ]:
    if len(ret.columns) >= 4: # if there are no columns set vals as NaN
        results_2_3["i"].append(i)  # First result is created outside of loop
        results_2_3["N"].append(len(ret.columns))  # Append Number of columns to results dictionary

In [ ]:
        # Sharpe ratio (for holding period)
        Sharpe_Ratio = ret.mean() / ret.std()  # Calculate sharpe ratio for each currency
        #Holding_SR = (126 ** 0.5) * Sharpe_Ratio  # Account for holding period

In [ ]:
        sum(Sharpe_Ratio/sum(Sharpe_Ratio))

In [ ]:
        weight = np.array(1 / len(ret.columns))  # Calculate 1/N weights
        weight_array = np.full((len(ret.columns), 1), weight)  # Create array with weight N times

In [ ]:
        opts = sco.minimize(min_func_sharpe, weight_array, method='SLSQP', bounds=bnds, constraints=cons)
        weight_array = opts["x"].T

In [ ]:
        # Portfolio return, Portfolio volatility, and Sharpe ratio
        # Add to array (for ex2.3 c and ex2.3 d)
        results_2_3["portfolio_return"].append(port_ret(weight_array))
        results_2_3["portfolio_volatility"].append(np.sum(port_vol(weight_array)))
        results_2_3["sharpe_ratio"].append(port_ret(weight_array) / np.sum(port_vol(weight_array)))
    else:
        i = i-1
        continue

### 3.3.d)

In [ ]:
#ex3 = plt.plot(results_2_3["N"], results_2_3["portfolio_volatility"])

## 4.)

In [ ]:
#%% Excersice 2.4
#fig, ex4 = plt.subplots(2)
#fig.suptitle('Comparing results from 2. and 3.')
#ex4[0].ex2
#ex4[1].ex3

print("The end")